In [1]:
import numpy as np
import pandas as pd
import torch
from scipy.linalg import toeplitz
from scipy.stats import norm

import matplotlib.pyplot as plt
from tableone import TableOne
from sksurv.nonparametric import kaplan_meier_estimator

from utils import data_processing, visualization
from utils.simulations import *
from execute import surv_hivae
#from utils.metrics import log_rank, cox_estimation

In [2]:
def simulation(beta_features, treatment_effect , n_samples , independent = True, surv_type = 'surv_piecewise', n_features_bytype = 4, 
                n_features_multiplier = 3, nnz = 3 , p_treated = 0.5,a_T=2,
                a_C = 2., lamb_C = 6., lamb_C_indpt = 2.5, data_types_create = True):
    n_features = n_features_multiplier * n_features_bytype
    beta = np.insert(beta_features, 0, treatment_effect)
    X = features_normal_cov_toeplitz(n_samples,n_features)
    X[:,(n_features_bytype ) : (2*n_features_bytype )] = np.abs(X[:,(n_features_bytype ) : (2*n_features_bytype )])
    X[:,(2*n_features_bytype ) : (3*n_features_bytype )] = 1 * (X[:,(2*n_features_bytype ) : (3*n_features_bytype )]>= 0)
    treatment = np.random.binomial(1, p_treated, size=(n_samples,1))
    design = np.hstack((treatment,X))
    marker = np.dot(design,beta)
    U = np.random.uniform(size = n_samples)
    V = np.random.uniform(size = n_samples)
    T = (- np.log(1-U) / np.exp(marker))**(1/a_T)
    if independent:
        C = lamb_C * (- np.log(1-V))**(1/a_C)
    else:
        C = lamb_C_indpt * (- np.log(1-V) / np.exp(marker))**(1/a_C)
    data = pd.DataFrame(X)
    data['treatment'] = treatment
    data['time'] = np.min([T,C],axis=0)
    data['censor'] = np.argmin([C,T],axis=0)
    control = data[data['treatment'] == 0]
    treated = data[data['treatment'] == 1]
    if data_types_create == True:
        names = []
        for x in range(1, n_features_bytype  * n_features_multiplier + 1):
            names.append("feat{0}".format(x))
        names.append("survcens")
        types = np.concatenate([np.repeat("real",n_features_bytype),np.repeat("pos",n_features_bytype),np.repeat("cat",n_features_bytype)]).tolist()
        types.append(surv_type)
        dims = np.repeat(1,n_features_bytype * n_features_multiplier).tolist()
        dims.append(2)
        nclasses = np.concatenate([np.repeat("",n_features_bytype),np.repeat("",n_features_bytype),np.repeat("2",n_features_bytype)]).tolist()
        nclasses.append("")
        data_types = pd.DataFrame({'name' : names , 'type' : types , 'dim' : dims, 'nclass' : nclasses})
        return(control,treated,data_types)
    else :
        return(control,treated)


In [3]:
n_samples = 600
n_features_bytype = 4
n_features_multiplier = 3 

In [7]:
beta_features = np.concatenate([weights_sparse_exp(n_features_bytype,3),weights_sparse_exp(n_features_bytype,3),
                       weights_sparse_exp(n_features_bytype,3)])
treatment_effect = 0.7

## Independent

In [8]:
control, treated, types = simulation(beta_features, treatment_effect , n_samples, independent = True, surv_type = "surv_weibull",
                                     n_features_multiplier = 3, nnz = 3 , p_treated = 0.5, a_T = 2, a_C = 1, lamb_C = 0.5, 
                                     data_types_create = True)
control = control.drop(columns='treatment')
treated = treated.drop(columns='treatment')

In [9]:
print(np.mean(control['censor']),np.mean(treated['censor']))

0.1362126245847176 0.2040133779264214


## Dependent

In [6]:
control, treated, types = simulation(beta_features, treatment_effect , n_samples, independent = False, surv_type = "surv_weibull",
                                     n_features_multiplier = 3, nnz = 3 , p_treated = 0.5, a_T = 2, a_C = 1, lamb_C = 0.15, 
                                     data_types_create = True)
control = control.drop(columns='treatment')
treated = treated.drop(columns='treatment')

In [7]:
print(np.mean(control['censor']),np.mean(treated['censor']))

0.13953488372093023 0.12709030100334448


## Compute expected power / level via Schoenfeld formula

$$D = \frac{(\Phi^{-1}(\beta)+\Phi^{-1}(1-\alpha))^2}{P_{cont}(1 - P_{cont}) log^2(\Delta)}$$
where 
- $D$ is the number of deaths
- $\alpha$ is the level
- $\beta$ is the power
- $P_{cont}$ is the proportion of patients in the control arm
- $\Delta$ is the hazard ratio

D

In [14]:
data = pd.concat([control,treated],ignore_index=True)
D = np.sum(1-data['censor'])
print(D)

520


In [15]:
alpha = 0.05
p_treated = 0.5
treatment_effect = 0.5
expected_power = norm.cdf(np.sqrt( D * p_treated * (1 - p_treated) * (treatment_effect)**2 ) - norm.ppf(1 - alpha/2))

In [16]:
expected_power

0.999908323558307